# tsne plots for phrase embeddings

I should read more about plotting wordembeddings:
http://learningaboutdata.blogspot.de/2014/06/plotting-word-embedding-using-tsne-with.html

I am using the barnes-hut python wrapper tsne.
The barnes-hut implementation has O(n log_n) complexity (the original algorithm uses O(n²).)

moreover I should know about how to render word-label with matplotlib
http://stackoverflow.com/questions/22859648/creating-a-wordcloud-with-matplotlib

In [34]:
import hickle
import numpy
from matplotlib import pyplot
from bokeh.plotting import figure, HBox, output_file, show, VBox
from bokeh.models import Range1d
from bokeh.charts import Scatter, output_file, show
import pandas as pd
import seaborn as sns
from gensim.models import Word2Vec

In [6]:
# tsne representation and word2vec model
penta_emb_tsne = hickle.load('/home/kuhn/Data/models/penta_tsne')
penta_model = Word2Vec.load('/home/kuhn/Data/models/pentagram_embeddings')

In [3]:
# Create a set of tools to use
TOOLS="pan,wheel_zoom,box_zoom,reset,previewsave"

In [6]:
phrasemb_plot

6540784

In [33]:

# give me just the first 100 values....
pyplot.scatter(phrasemb_plot[:100, 0], phrasemb_plot[:100, 1], s=10)

In [34]:
# this works out pretty nice, still missing word-labels though 
pyplot.show()

In [45]:
# this is the dimensionality of the array:
phrasemb_plot.shape

(408799, 2)

In [59]:
pyplot.scatter(phrasemb_plot[1, 0], phrasemb_plot[1, 1], s=10)

In [7]:
pyplot.show()

In [61]:
# I need a csv file with xy coords of each word

### Labelling Words to the Scatterplot Coords

First, take a look at a snippet of the tutorial on Jane Austen's Pride and Prejude with Word Embedding (thx to Arne for the advice)



```Python
# The wrapper function that runs everything, writes files, and returns useful things.
def run_all(tags=['NN'], filepref='data/pride_NN'):
    
    # notice here i require at least 2 uses of each word in the corpus - reduces noise
    
    # uncomment the version of the model you want to try: tag limited, or all words.
    #model = gensim.models.Word2Vec(MySentences('books', tags=tags), min_count=2, size=200, workers=2)
    model = gensim.models.Word2Vec(DirOfPlainTextCorpus('books'), min_count=3, workers=2)
    model.save(filepref + '_model_austen_allwordsmin3')
    
    # if you want to save time and use a saved file, comment out the above and uncomment this with right path
    #model = gensim.models.Word2Vec.load('data/pride_NNPRP_model_austen_all')
    
    # does the text tagging and word replacement
    print tags
    datadict = build_dict_write_file(model, filepref + '_labeled.txt', filepref + '_data.json', tags=tags)
    
    # tsne input files part
    make_score_files(model, datadict, filepref)
    
    # the actual tsne graph bit
    X = np.loadtxt(filepref + "_scores.csv")
    labels = np.genfromtxt(filepref + "_words.csv", dtype=str)
    Y = tsne.tsne(X, 2, 50, 20.0)   # see tsne.py in repo
    do_tsne_files(filepref + '_coords.tsv', Y, labels, datadict, axis_off=True)
    
    return Y, labels, datadict, model
```


first, the w2v model ist trained - a goal we already achieved. the model is then saved to a file.
the 'datadict' line. Section *#the actual tsne graph bit* pretty much contains the info we need.
the line
```python
labels = np.genfromtxt(filepref + "_words.csv", dtype=str)
```

takes a csv file that contains the vocabulary of the corpus and konverts it to a numpy array.

We already have calculated the tsne representation of our phrase embedding so it is important to take a look at this line
```python
do_tsne_files(filepref + '_coords.tsv', Y, labels, datadict, axis_off=True)
```
the corresponding method is defined below:

```Python

# this is a function you can call independently with output from your run-all run, to jigger with details if you want.
def do_tsne_files(coordsfilename, Y, labels, datadict, axis_off=True):
    """ Makes the tsne grah file and writes out the coords for the UI. """
    plt.figure(figsize=(15, 15))
    if axis_off:
        plt.axis('off')   # only turn off after checking the axis min/max
    plt.scatter(Y[:,0], Y[:,1], s=10, color='gray', alpha=0.2)
    print "X and Y dims ", plt.axis()
    with open(coordsfilename, 'w') as handle1:
        handle1.write('word\tx\ty\tnearest\n')
        for label, x, y in zip(labels, Y[:, 0], Y[:, 1]):
            score = datadict[label]['score']
            handle1.write('\t'.join([label, str(x), str(y), str(score)]) + '\n')
            #plt.text(x, y, label, size=8, alpha=0.5)
    print "wrote out", coordsfilename
```




we get a list all indexed words of the model via the index2word attribute. 

In [8]:
penta_model.index2word[0]

u'angeforderten_Gerichtskosten'

In [13]:
numpy.array?

we can use numpy.array to turn the list to a numpyarray.
http://docs.scipy.org/doc/numpy/user/basics.creation.html

In [15]:
labels = numpy.array(penta_model.index2word)

now modifying do_tsne_files to fit our needs (w/o score parameter). using codecs to deal with unicode.

In [61]:
import codecs

def do_tsne_files(coordsfilename, Y, labels, axis_off=True):
    """ Makes the tsne grah file and writes out the coords for the UI. """
    plt.figure(figsize=(15, 15))
    if axis_off:
        plt.axis('off')   # only turn off after checking the axis min/max
    plt.scatter(Y[:,0], Y[:,1], s=10, color='gray', alpha=0.2)
    print "X and Y dims ", plt.axis()
    with codecs.open(coordsfilename, mode='w', encoding='utf-8' ) as handle1:
        handle1.write('word\tx\ty\n')
        for label, x, y in zip(labels, Y[:, 0], Y[:, 1]):
            
            handle1.write('\t'.join([label, str(x), str(y)]) + '\n')
            #plt.text(x, y, label, size=8, alpha=0.5)
    print "wrote out", coordsfilename

In [62]:
from matplotlib import pyplot as plt 
do_tsne_files('/tmp/misc.tsv', penta_emb_tsne, labels)

X and Y dims  (-4.0, 4.0, -4.0, 4.0)
wrote out /tmp/misc.tsv


somehow we expected the german language to spoil the party. lets deal with numpy to unicode tasks
http://stackoverflow.com/questions/16037824/how-to-convert-numpy-object-array-into-str-unicode-array